# Implementing a CIC filter using C 

When implementing a filter on a microprocessor, we generally need three things:
1. State variables which can retain information about the filter state between filtering operations.
2. A way to initialize filter states.
3. A function to process the input data $x[n]$ and calculate the output data $y[n]$. This could either be done on a sample-by-sample basis, or using entire arrays as input.


If we were programming in C++, Java, Python or any other object oriented language, all of the above could be wraped int one class. However, since we are programing the stm32 in C, we will have to make do with the next-best thing: structs and functions.

In this task we will have a closer look at one possible implementation of a CIC filter, where we process an entire array of input data (such as the array `ADC_buffer` from assignment $6$). The structure of the solution is closely modeled on similar functions in the CMSIS-DSP library we used to perform the RFFT in assignment $6$.

#### 1. Defining a struct for storing information about filter state

The following code declares a struct with three member variables. Think of this as the private variables of a class. The struct is given an identifier `CIC_isnt_int16` which allows us to pass struct pointer as arguments to functions.

```C
typedef struct
{
	uint16_t decimation_factor;
	int32_t acc;
	int32_t prev_acc;

}CIC_inst_int16;
```

#### 2. Initializing the filter state
Below is a function description for a function `CIC_int16_init` which initializes the filter state, and sets crucial filter parameters such as decimation factor. Think of this as the filter's constructor.

```C
void CIC_int16_init(CIC_inst_int16 *S, uint16_t M){
	S->acc = 0;
	S->prev_acc = 0;
	S->decimation_factor = M;
}
```

#### 3. Downsampling signal by processing arrays
Below is a function description for a function `CIC_int16` which is responsible for doing the actual downsampling.
```C

void CIC_int16(CIC_inst_int16 *S,
		int16_t *input,
		uint32_t n_samples,
		int16_t *output)
{
	uint32_t n=0;
	uint32_t m=0;
	while(n<n_samples){
		S->acc += input[n];
        n++;
		if (n%S->decimation_factor==0){
			output[m] = (uint16_t)(S->acc - S->prev_acc);
			m++;
			S->prev_acc = S->acc;
		}
	}
}
```

## a)
* Study the code above in steps 1-3, and try to understand what is going on in each part. Write a suggestion for how you would use these function/struct definitions to downsample data in an array (e.g. `ADC_buffer`) by a factor of `8`, and submit your answer as an abbreviated `void main()` function description. (You don't need to include all the `HAL_ADC_Init`-calls etc., only the parts relevant to this downsampling operation specifically.

ANSWER THEORY QUESTIONS HERE:

```C
// Step 1: Declare the instance of the struct
    CIC_inst_int16 cic_filter;

// Step 2: init the filter, with a factor of 8
    CIC_int16_init(&cic_filter, 8);

// Step 3: Prepare I/O buffers
    int16_t ADC_buffer[N_SAMPLES]; 
    int16_t downsampled_buffer[N_SAMPLES / 8];


    CIC_int16(&cic_filter, ADC_buffer, N_SAMPLES, downsampled_buffer);
```